<a href="https://colab.research.google.com/github/DAUN3046/AI-Agent-Exercise/blob/main/Colab_CrewAI_%EC%8B%A4%ED%96%89%EC%98%88%EC%A0%9C_togetherai%EC%82%AC%EC%9A%A9_%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CrewAI 실행예제

CrewAI 실습 후기 공유드립니다.

- 회사 노트북으로 주피터를 쓰려니 cmd 대부분의 기능이 보안프로그램으로 차단되어서 구글 코랩을 사용했습니다.
- OpenAI API는 선결제가 필요하여 무료 크레딧을 주는 TogetherAI API를 이용했고,
- `Meta-Llama-3-8B-Instruct-Turbo` 모델을 선택했습니다.(익숙한 모델명이어서 택한거고 다른 모델도 많아서 실습하실 땐 liteLLM에서 값싼 모델로 돌리면 되겠습니다.)

결론적으로 기사 써보라고 `목차 설계자`와 `본문 작성자`를 둬서 시켜보니 씁니다.

삽질했던 부분은 아래와 같습니다.
1. crewai가 litellm을 사용하여 LLM들에 연결되기 때문에 트랜스포머를 이용해 로컬 모델에 접근하는건 불가능합니다(litellm 목록 내 모델만 사용 가능).
2. Provider 제공 오류는 모델 경로를 상세히 지정해주어야 됩니다. 예를 들어 togherai를 사용할 경우 provider에 togherai를 적고 model에는 모델명만 적었는데 이러면 연결이 안되고, model 경로에도 최상단에 제공자명을 포함해야 됩니다.

# 패키지 설치

In [ ]:
# CrewAI 설치
!pip install crewai transformers accelerate --quiet
!pip install crewai litellm duckduckgo-search
!pip install together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.1 MB/s eta 0:00

# 모델 로딩

In [ ]:
from crewai import Agent, Task, Crew, LLM
import os
# ✅ 0. 환경변수로 Together API 키 설정 (선택)
os.environ["TOGETHERAI_API_KEY"] = ""

In [ ]:
# from together import Together

# client = Together(api_key="")

# response = client.chat.completions.create(
#     model="meta-llama/Meta-Llama-3-8B-Instruct-Turbo",
#     messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
# )
# print(response.choices[0].message.content)

The city that never sleeps! New York City is a vibrant and exciting place to visit, with endless options for entertainment, culture, and adventure. Here are some fun things to do in New York:

**Iconic Landmarks:**

1. Visit the Statue of Liberty and Ellis Island: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.
2. Explore the Empire State Building: Enjoy panoramic views of the city from the observation deck on the 86th floor.
3. Walk across the Brooklyn Bridge: Take in the stunning views of the Manhattan skyline and the East River.

**Museums and Galleries:**

1. The Metropolitan Museum of Art: One of the world's largest and most famous museums, with a collection of over 2 million works of art.
2. The Museum of Modern Art (MoMA): A must-visit for modern art enthusiasts, with a collection of works by artists like Van Gogh, Picasso, and Warhol.
3. The Guggenheim Museum: Designed by Frank Lloyd Wright, this museum features an

# 에이전트 및 작업 정의

In [ ]:
# ✅ 1. Together AI 기반 LLM 정의
llm = LLM(
    model="together_ai/meta-llama/Meta-Llama-3-8B-Instruct-Turbo",  # Together에서 지원하는 고성능 LLM
    provider="together_ai",
    config={
        "temperature": 0.7,
        "max_tokens": 512,
        "api_key": ""
    }
)

# ✅ 2. 에이전트 정의
outline_agent = Agent(
    role="목차 설계자",
    goal="주어진 주제에 대해 논리적인 기사 목차를 작성합니다. 반드시 한국어로 작성합니다.",
    backstory="정보 구조화에 능한 편집 전문가입니다.",
    llm=llm
)

writer_agent = Agent(
    role="본문 작성자",
    goal="목차를 참고하여 각 항목에 대한 본문을 작성합니다. 반드시 한국어로 작성합니다.",
    backstory="설득력 있고 쉽게 읽히는 콘텐츠를 잘 작성하는 작가입니다.",
    llm=llm
)

# ✅ 3. Task 정의
task1 = Task(
    description="제로트러스트 보안 전략에 대한 기사 목차를 작성해줘.",
    expected_output="1. 서론, 2. 제로트러스트의 개념, 3. 핵심 구성 요소, 4. 적용 사례 등",
    agent=outline_agent
)

task2 = Task(
    description="task1에서 작성한 목차를 바탕으로 각 항목에 대한 본문을 작성해줘.",
    expected_output="각 항목에 대한 2~3문장 이상의 설명으로 구성된 본문",
    agent=writer_agent
)

# ✅ 4. Crew 정의
crew = Crew(
    agents=[outline_agent, writer_agent],
    tasks=[task1, task2],
    verbose=True
)

# 실행

In [ ]:
if __name__ == "__main__":
    try:
        result = crew.kickoff()
        print("\n📝 최종 결과:\n", result)
    except Exception as e:
        print(f"An error occurred: {e}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 547aba16-d51f-40cd-9be1-9bde81d0eff9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 551bdc24-afbf-4bd5-8a3b-a1c3d134e472
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 551bdc24-afbf-4bd5-8a3b-a1c3d134e472
       Status: Executing Task...
    └── 🤖 Agent: 목차 설계자
            Status: In Progress

# Agent: 목차 설계자
## Task: 제로트러스트 보안 전략에 대한 기사 목차를 작성해줘.


🚀 Crew: crew
└── 📋 Task: 551bdc24-afbf-4bd5-8a3b-a1c3d134e472
       Status: Executing Task...
    └── 🤖 Agent: 목차 설계자
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 551bdc24-afbf-4bd5-8a3b-a1c3d134e472
│      Assigned to: 목차 설계자
│      Status: ✅ Completed
│   └── 🤖 Agent: 목차 설계자
│           Status: ✅ Completed
└── 📋 Task: c3237d62-bd36-4ee6-b2d6-af587073ff58
       Status: Executing Task...
    └── 🤖 Agent: 본문 작성자
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 551bdc24-afbf-4bd5-8a3b-a1c3d134e472
│      Assigned to: 목차 설계자
│      Status: ✅ Completed
│   └── 🤖 Agent: 목차 설계자
│           Status: ✅ Completed
└── 📋 Task: c3237d62-bd36-4ee6-b2d6-af587073ff58
       Assigned to: 본문 작성자
       Status: ✅ Completed
    └── 🤖 Agent: 본문 작성자
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c3237d62-bd36-4ee6-b2d6-af587073ff58                                                                     │
│  Agent: 본문 작성자                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 547aba16-d51f-40cd-9be1-9bde81d0eff9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


📝 최종 결과:
 I. 서론
제로트러스트(Zero Trust) 보안 전략은 네트워크 보안에서 새로운 접근 방식으로, 모든 네트워크 트래픽을 검증하고 인증하는 방식입니다. 이 기사에서는 제로트러스트의 개념, 핵심 구성 요소, 적용 사례 등을 다룹니다. 제로트러스트는 네트워크 보안에서 새로운 접근 방식으로, 모든 네트워크 트래픽을 검증하고 인증하는 방식입니다. 이러한 방식에서는 네트워크 내부의 모든 장비를 신뢰할 수 없으며, 모든 트래픽을 검증하고 인증해야 합니다.

II. 제로트러스트의 개념
제로트러스트는 네트워크 보안에서 새로운 접근 방식으로, 모든 네트워크 트래픽을 검증하고 인증하는 방식입니다. 이 방식에서는 네트워크 내부의 모든 장비를 신뢰할 수 없으며, 모든 트래픽을 검증하고 인증해야 합니다. 이러한 방식에서는 네트워크 내부의 모든 장비를 신뢰할 수 없으며, 모든 트래픽을 검증하고 인증해야 합니다.

III. 핵심 구성 요소
제로트러스트의 핵심 구성 요소는 다음과 같습니다. 첫째, 네트워크 트래픽 검증: 모든 네트워크 트래픽을 검증하여 악성 코드나 스파이웨어를 차단합니다. 둘째, 인증: 모든 장비를 인증하여 네트워크 내부의 모든 장비를 신뢰할 수 없습니다. 셋째, 액세스 제어: 액세스 제어를 통해 네트워크 내부의 모든 장비에 대한 액세스를 제어합니다.

IV. 적용 사례
제로트러스트는 다양한 산업에서 적용할 수 있습니다. 첫째, 금융 industry: 제로트러스트를 사용하여 금융 정보를 보호할 수 있습니다. 둘째, 보건 industry: 제로트러스트를 사용하여 보건 정보를 보호할 수 있습니다. 셋째, 정부 industry: 제로트러스트를 사용하여 정부 정보를 보호할 수 있습니다.

V. 결론
제로트러스트 보안 전략은 네트워크 보안에서 새로운 접근 방식으로, 모든 네트워크 트래픽을 검증하고 인증하는 방식입니다. 이 방식에서는 네트워크 내부의 모든 장비를 신뢰할 수 없으며, 모든 트래픽을 검증하고 인증해야 합니다. 제로트러스트는 다양한 산업에서 적용할 수 